In [1]:
%pdb

Automatic pdb calling has been turned ON


In [2]:
from sleepmate.agent import *

In [ ]:
!rm -f /Users/cck197/Downloads/virtual_greg/memory.pkl

In [2]:
from sleepmate.helpful_scripts import *

In [3]:
imported = import_attrs(["GOALS", "GOAL_HANDLERS"])

In [3]:
imported = import_attrs(["TOOLS"])

In [4]:
imported["TOOLS"]

[<function sleepmate.isi.get_last_isi_entry(memory: langchain.memory.readonly.ReadOnlySharedMemory, goal: str, utterance: str)>,
 <function sleepmate.isi.get_date_isi_diary_entry(memory: langchain.memory.readonly.ReadOnlySharedMemory, goal: str, utterance: str)>,
 <function sleepmate.isi.get_isi_dates(memory: langchain.memory.readonly.ReadOnlySharedMemory, goal: str, utterance: str)>,
 <function sleepmate.isi.save_isi_entry(memory: langchain.memory.readonly.ReadOnlySharedMemory, goal: str, utterance: str)>,
 <function sleepmate.diary.get_sleep_diary_description(memory: langchain.memory.readonly.ReadOnlySharedMemory, goal: str, utterance: str) -> str>,
 <function sleepmate.diary.get_last_sleep_diary_entry(memory: langchain.memory.readonly.ReadOnlySharedMemory, goal: str, utterance: str)>,
 <function sleepmate.diary.get_date_sleep_diary_entry(memory: langchain.memory.readonly.ReadOnlySharedMemory, goal: str, utterance: str)>,
 <function sleepmate.diary.get_sleep_diary_dates(memory: langc

In [5]:
flatten_list_of_dicts(imported)

ValueError: dictionary update sequence element #0 has length 1; 2 is required

> /Users/cck197/Documents/sleepmate/sleepmate/helpful_scripts.py(62)flatten_list_of_dicts()
     60     result = {}
     61     for d in dicts_list:
---> 62         result.update(d)
     63     return result
     64 



ipdb>  exit


In [5]:
imported_ = flatten_dict(imported)

In [6]:
imported_.keys()

dict_keys(['GOALS', 'GOAL_HANDLERS'])

In [7]:
imported_["GOAL_HANDLERS"]

{'insomnia_severity_index': <function sleepmate.isi.insomnia_severity_index()>,
 'diary_probe': <function sleepmate.diary.diary_probe()>,
 'health_history': <function sleepmate.history.health_history()>}

In [8]:
imported_

{'GOALS': {'insomnia_severity_index': "\n        Your goal is to survey the human for their Insomnia Severity Index (ISI)\n        using the standard 7-item questionnaire, but don't ask until you've\n        asked an open question, and the human has confirmed the accuracy of at\n        least one listening statement. Ask if now would be a good time then the\n        following questions. Don't ask more than one question at time.\n        \n        0. Date of entry (get today's date and make that the default)\n        1. Difficulty falling asleep.\n        2. Difficulty staying asleep.\n        3. Problems waking up too early.\n        4. How satisfied/dissatisfied are you with your current sleep pattern?\n        5. How noticeable to others do you think your sleep problem is in terms\n           of impairing the quality of your life?\n        6. How worried/distressed are you about your current sleep problem?\n        7. To what extent do you consider your sleep problem to interfere wit